### Import Modules

In [ ]:
import os
import numpy as np
import pandas as pd
from functools import reduce
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel, mutual_info_regression
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_absolute_error
from tpot import TPOTRegressor
import shap
import warnings
warnings.filterwarnings('ignore')

### Read Training Data

In [ ]:
train_data_path = 'train'
predictors_paths = {
    'GM': os.path.join(train_data_path, 'GM.csv'),
    'WM': os.path.join(train_data_path, 'WM.csv'),
    'ReHo': os.path.join(train_data_path, 'ReHo.csv'),
    'PCGcorr': os.path.join(train_data_path, 'PCGcorr.csv'),
    'FA': os.path.join(train_data_path, 'FA.csv'),
    'MD': os.path.join(train_data_path, 'MD.csv')
}
additional_variables_path = os.path.join(train_data_path, 'Subjects.csv')

# Predictors
modalities = ['GM', 'ReHo', 'MD']
selected_modalities = {modality: predictors_paths[modality] for modality in modalities if modality in predictors_paths}
def read_and_rename(modality, path):
    df = pd.read_csv(path)
    df = df.rename(columns={label: f"{label}_{modality}" for label in df.columns if label != 'ID'})
    return df
dfs = [read_and_rename(modality, path) for modality, path in selected_modalities.items()]
predictors_df = reduce(lambda left, right: pd.merge(left, right, on='ID'), dfs)

# Response and confounding variables
additional_variables = ['Memory', 'Age', 'Sex', 'EducationYear']
df = pd.read_csv(additional_variables_path)
selected_variables = [variable for variable in additional_variables if variable in df.columns]
additional_variables_df = df[["ID"] + selected_variables]

# Merge predictors with response and confounding variables on 'ID'
df = pd.merge(additional_variables_df, predictors_df, on='ID')

# Prepare X and y
X = df.drop(columns=['ID', 'Memory'])
y = df['Memory']

# Split into training and test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(f'Sample size for training: {X_train.shape[0]}')
print(f'Sample size for test: {X_test.shape[0]}')

### AutoML: TPOT (Tree-based Pipeline Optimization Tool)
$ \text{Total pipelines evaluated} = \text{population size} + (\text{generations} \times \text{offspring size}) $

In [ ]:
# http://epistasislab.github.io/tpot/using/
# generations: number of iterations to run the pipeline optimization process
# population_size: number of individuals to retain in the genetic programming (GP) population every generation
# offspring_size: number of offspring to produce in each GP generation; by default, offspring_size = population_size
# total pipelines evaluated = 50 + (5 x 50) = 300
tpot = TPOTRegressor(generations=5, population_size=50, verbosity=2, random_state=42, scoring='neg_mean_absolute_error')

### Train and Test Model

In [ ]:
tpot.fit(X_train, y_train)
predictions = tpot.predict(X_test)
mae = mean_absolute_error(y_test, predictions)
print(f"MAE by TPOT: {mae:.3f}")
best_pipeline = tpot.fitted_pipeline_
print("Best pipeline steps:")
for step in best_pipeline.steps:
    print(step)
tpot.export('tpot_best_pipeline.py')

### Feature Importances

In [ ]:
regressor = best_pipeline.steps[-1][1]
features = X_train.columns if isinstance(X_train, pd.DataFrame) else range(X_train.shape[1])
if hasattr(regressor, 'feature_importances_'):
    feature_importances = regressor.feature_importances_
elif hasattr(regressor, 'coef_'):
    feature_importances = regressor.coef_
features_and_importances = zip(features, feature_importances)
sorted_features_and_importances = sorted(features_and_importances, key=lambda x: x[1], reverse=True)
top_features_and_importances = sorted_features_and_importances[:9]
print(f"Top features' importances by TPOT:")
for no, feature_importance in enumerate(top_features_and_importances):
    print(f"{no + 1}. {feature_importance[0]}: {feature_importance[1]:.3f}")

### SHAP (SHapley Additive exPlanations)

In [ ]:
explainer = shap.Explainer(regressor, X_train)
shap_values = explainer(X_test) # (16, 171)
shap.initjs()
shap.summary_plot(shap_values, X_test, max_display=9)

### Inference

In [ ]:
test_data_path = 'test'
predictors_paths = {
    'GM': os.path.join(test_data_path, 'GM.csv'),
    'WM': os.path.join(test_data_path, 'WM.csv'),
    'ReHo': os.path.join(test_data_path, 'ReHo.csv'),
    'PCGcorr': os.path.join(test_data_path, 'PCGcorr.csv'),
    'FA': os.path.join(test_data_path, 'FA.csv'),
    'MD': os.path.join(test_data_path, 'MD.csv')
}
additional_variables_path = os.path.join(test_data_path, 'Subjects.csv')

# Predictors
selected_modalities = {modality: predictors_paths[modality] for modality in modalities if modality in predictors_paths}
dfs = [read_and_rename(modality, path) for modality, path in selected_modalities.items()]
predictors_df = reduce(lambda left, right: pd.merge(left, right, on='ID'), dfs)

# Confounding variables
df = pd.read_csv(additional_variables_path)
selected_variables = [variable for variable in additional_variables if variable in df.columns]
additional_variables_df = df[['ID'] + selected_variables]

# Merge predictors with confounding variables on 'ID'
df = pd.merge(additional_variables_df, predictors_df, on='ID')

# Apply trained model
X_ext = df.drop(columns=['ID'])
predictions_ext = regressor.predict(X_ext)

# Save predictions
np.savetxt(os.path.join(test_data_path, "Predictions.txt"), predictions_ext)

# SHAP
shap_values = explainer(X_ext) # (10, 171)
shap.initjs()
shap.summary_plot(shap_values, X_ext, max_display=9)